In [ ]:
import requests
import pandas as pd

# Importing HPI data and Vaccine data
voter_data = requests.get('https://api.healthyplacesindex.org/api/hpi?geography=tracts&year=2020&indicator=voting&format=json&key=f07f86d6-58b5-40e3-a17f-333b92ccc57d').json()
vaccine_data = requests.get('https://data.sandiegocounty.gov/resource/nrkb-eanb.json').json()

# voter_data
# vaccine_data

# Converting data (python dictionary) to a pandas dataframe
# Resetting titles (variables) the dataframe versions
voter_data = pd.DataFrame(voter_data)
voter_data['geoid'] = voter_data['geoid'].str[1:].astype({'geoid':'int64'})

vaccine_data = pd.DataFrame(vaccine_data)

# voter_data.head()

# len(voter_data)

In [ ]:
# Converting Vaccine Data column to float
vaccine_data['vaccination_percentage_all'] = pd.to_numeric(vaccine_data['vaccination_percentage_all'])
# vaccine_data.dtypes

# Creating a function to popuilated a column
import numpy as np

def Vaccine_Quartile (vaccination_percentage_all):
    if vaccination_percentage_all <= 25:
        return 'Q1'
    elif vaccination_percentage_all <= 50:
        return 'Q2'
    elif vaccination_percentage_all <= 75:
        return 'Q3'
    elif vaccination_percentage_all <= 100:
        return 'Q4'
    else:
        return 'Unknown'
    
vaccine_data['Vaccine Quartile'] = vaccine_data.apply(lambda row: Vaccine_Quartile(row['vaccination_percentage_all']), axis=1)

vaccine_data.head(2)

In [ ]:
#Import from Google Drive - Master HPI Data. Recieved this file directly from the HPI agency. 
# Will use this file as an intermidiary- common key is needed from this file to join HPI data and Vaccine data

import pandas as pd

url='https://drive.google.com/file/d/1-O5aXOkgAaQUxMjZ5EXu9y6kHxmjsyp2/view?usp=share_link'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
master_hpi = pd.read_csv(path)

master_hpi = pd.DataFrame(master_hpi)

#Cleaning/Filtering mster HPI data. Preparing to join. 
# san_diego_hpi = master_hpi[master_hpi['county']=='San Diego']
san_diego_hpi = master_hpi[master_hpi['county']=='San Diego']


san_diego_hpi['NAME'] = [x.split(',')[-0] for x in san_diego_hpi['NAME']]

san_diego_hpi['NAME'] = [x.split('Tract ')[-1] for x in san_diego_hpi['NAME']]

# san_diego_hpi.head(2)
# len(master_hpi)


In [ ]:
# Joining datasets
#san diego hpi: vaccine data

sdhpi_vaccine_join = pd.merge(san_diego_hpi[['voting','voting_pctile','NAME','GEO_ID','county','pop','hpi_pctile','hpi_quartile','hpi_least_healthy_25pct','economic_pctile']], vaccine_data[['census_tract','gis_hpi_quartile_sd', 'vaccination_percentage_all','Vaccine Quartile', 'update_date']], left_on='NAME', right_on='census_tract', how='left')

sdhpi_vaccine_join.sort_values('vaccination_percentage_all', ascending=False).drop_duplicates(['voting','voting_pctile','NAME','GEO_ID','county','pop','hpi_pctile','hpi_quartile','hpi_least_healthy_25pct','economic_pctile','census_tract','gis_hpi_quartile_sd', 'Vaccine Quartile']).sort_index()
# test_join[test_join['census_tract'] == '170.19' ]

# sdhpi_vaccine_join

# voter_data.head(2)


In [ ]:
# Merging all 3 datasets together
three_set_join = pd.merge(sdhpi_vaccine_join[['voting','voting_pctile','NAME','GEO_ID','county','pop','hpi_pctile','hpi_quartile','hpi_least_healthy_25pct','economic_pctile','census_tract','gis_hpi_quartile_sd', 'vaccination_percentage_all','Vaccine Quartile']],voter_data[['geoid','name','population','value', 'percentile']], left_on='GEO_ID', right_on='geoid', how='left')
three_set_join


In [ ]:
# Selected Columns - Reduced to specific columns
select_df = three_set_join[['voting', 'voting_pctile', 'GEO_ID', 'hpi_pctile', 'hpi_quartile','Vaccine Quartile', 'vaccination_percentage_all']]
select_df

In [ ]:
# # Writing to CSV files for all three datasets

# from pathlib import Path 

# # Export of Filtered San Diego HPI data (sample size) - Accessed via API
# export_sdhpi = san_diego_hpi.head(10)
# filepath = Path('C:/Users/krist/Downloads/DSCI-510-Final-Project/san_diego_hpi_sample.csv')
# filepath.parent.mkdir(parents=True, exist_ok=True)
# export_sdhpi.to_csv(filepath)


# # Export of vaccine data (sample size) - accessed via json import
# export_v = vaccine_data.head(10)
# filepath = Path('C:/Users/krist/Downloads/DSCI-510-Final-Project/vaccine_data_sample.csv')
# filepath.parent.mkdir(parents=True, exist_ok=True)
# export_v.to_csv(filepath)

# # Export of Master HPI data - reading from a Google Drive folder
# export_master = vaccine_data.head(10)
# filepath = Path('C:/Users/krist/Downloads/DSCI-510-Final-Project/master_data_sample.csv')
# filepath.parent.mkdir(parents=True, exist_ok=True)
# export_master.to_csv(filepath)

# #Export of merged dataset of above three imported datasets
# export_three_set = three_set_join.head(10)
# filepath_joined = Path('C:/Users/krist/Downloads/DSCI-510-Final-Project/Resulting Joined Data.csv')
# filepath_joined.parent.mkdir(parents=True, exist_ok=True)
# export_three_set.to_csv(filepath_joined)

In [ ]:
import plotly.express as px

three_set_join = three_set_join[three_set_join['Vaccine Quartile'] !='Unknown']
val_by_percentile = px.scatter(three_set_join, x='value', y='percentile', color='Vaccine Quartile', labels={'percentile':'HPI Percentile', 'value':'Vaccination Value'}, title='Distribution Plot (Cumulative): HPI Value by HPI Percentile Score for all Census Tracts<br><sup>This plot shows the directly proportional relationship between the HPI Percentiles and census tracts represented by administered vaccines (Value),<br>which are measured against their assigned HPI Percentile (ie.Score). This chart is merely to illustrate the directly positive correlation between the two categories.')
val_by_percentile.show()


In [ ]:
# Number of Census Tract by HPI Quartile

import plotly.express as px

cfinal_joined = sdhpi_vaccine_join.groupby('hpi_quartile').count().reset_index()

bar_chart = px.bar(cfinal_joined, y='census_tract', x='hpi_quartile', color='hpi_quartile', labels={'hpi_quartile':'HPI Quartile', 'census_tract':'Count of Census Tracts'}, title='Count of Census Tracts by HPI Quartile <br><sup>This chart provides a simple count of the number of census tracts for each HPI Quartile, which is an extension of the above scatter plot.')
bar_chart.show()
# bar_chart.write_image('Count of Census Tracts by HPI Quartile.pdf', engine = 'kaleido')


In [ ]:
# Pie Chart Visual using calculated column : Vaccine Quartile

pie_df = sdhpi_vaccine_join.groupby('Vaccine Quartile').count().reset_index()
pie_df = pie_df[pie_df['Vaccine Quartile'] != 'Unknown']

pie_chart = px.pie(pie_df, values='voting_pctile', names='Vaccine Quartile', color='Vaccine Quartile',
    title= 'Density of Voter Percentile by Vaccine Quartile<br><sup>The importance of this chart is to illustrate the density of each quartile for the range of Voter Percentiles (turnout success).<br>For instance, the purple section accounts for 54.4% of Voters that are in the highest-vaccinated percentile.')
pie_chart.show()
# pie_chart.write_image('Account of Vaccine Quartile Amount.pdf', engine = 'kaleido')


In [ ]:
#Scatter Plot - TSJ version
val_by_population_tsj = px.scatter(three_set_join, x='value', y='population',labels={'value':'Voter Value', 'population':'HPI Population'}, color='Vaccine Quartile', title='Population Clustering within Voter Value (turnout)<br><sup>This chart shows the commonality of Census Tracts by their assigned HPI Value. We see that nearly all populations per census tracts remain less than 10k people,<br>where most of those populations have a voter turnout between 0.6 - 0.9 percent. The majority of vaccinated people tend to have higher voter turnout.')
val_by_population_tsj.show()

In [ ]:
# import matplotlib.pyplot as plt
import plotly.express as px

xfinal_joined = sdhpi_vaccine_join[sdhpi_vaccine_join['vaccination_percentage_all'] != 99]

fig = px.scatter(xfinal_joined, x='vaccination_percentage_all', y='hpi_pctile', labels={'vaccination_percentage_all':'Vaccination Percentage', 'hpi_pctile':'HPI Percentile', 'gis_hpi_quartile_sd': 'HPI Quartile'}, color='gis_hpi_quartile_sd', title='Visual for Vaccination Percentage by HPI Percentile <br><sup>This plot shows the HPI Percentiles (by census tract) that are categorically clustered by their Vaccination percentages. The value of this chart allows us to compare all the categories to each other.<br>Each of the colored HPI Quartile illustrate proactive voting activity. As obviously shown, the most proactive voters are in the highest percentile and lacks clear definition of vaccination activity.')
fig.show()
# fig.write_image('Vaccination Percentage by HPI Percentile.pdf', engine = 'kaleido')


In [ ]:
# Comparable Scatter chart - More realistic and robust camparison

for_main_trend = three_set_join[three_set_join['vaccination_percentage_all'] != 99]
main_trend = px.scatter(for_main_trend, x='vaccination_percentage_all', y='voting_pctile', labels={'vaccination_percentage_all':'Vaccination Percentage', 'voting_pctile':'Voting Percentile (turnout success)', 'gis_hpi_quartile_sd': 'HPI Quartile'}, color='gis_hpi_quartile_sd', title='Visual for Vaccination Percentage by Voting Percentile<br><sup>This scatter plot is the result of the unification between the three sets of data. This plot more robustly illustrates the overlap of HPI Quartiles amongst voting activity.<br>We see that HPI Quartile 4 has the most defined clustering, having a Vaccination Percentage mainly between 70-90%. HPI Quartiles 3 and 4 are also distinctly mixed, whereas 1 and 2 are seemingly more defined.')
main_trend.show()
# main_trend.write_image('Vaccination Percentage by Voting Percentile.pdf', engine='kaleido', format='pdf')

# sdhpi_vaccine_join.dtypes

In [ ]:
# Visual - Original Distribution Curve (not used)

# val_by_percentile = plt.scatter(voter_data['value'],voter_data['percentile'], c='green', s=5)
# val_by_percentile = plt.xlabel('HPI Value')
# val_by_percentile = plt.ylabel('Percentile Score')
# val_by_percentile = plt.title('Distribution Plot (Cumulative): HPI Value by Percentile Score for all Census Tracts')
# val_by_percentile = plt.suptitle('Distribution Plot (Cumulative): HPI Value by Percentile Score for all Census Tracts')


# plt.show(val_by_percentile)
# plt.savefig('Distribution Plot (Cumulative)- HPI Value by Percentile Score for all Census Tracts.pdf') 

In [ ]:
#  Future Work: 3D - Economic Percentile Correlation with HPI Value

import matplotlib.pyplot as plt

val_by_population_3d = px.scatter_3d(three_set_join, x='value', y='economic_pctile', z='population', title='Population and Economic Clustering within HPI Value (Score)')
val_by_population_3d.show()



In [ ]:
import session_info
session_info.show()